<a href="https://colab.research.google.com/github/elliemci/chatbots/blob/main/med_ner_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medication Name Entity Recognition Chatbot extracting entities from medication texts

In [1]:
!pip install transformers pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215252 sha256=9f73699cf574a35006b0e3e19958f95f0647c0529d86b3c849b0633d13a80452
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
Successfully built pyTelegramBotAPI


In [23]:
import os
import telebot
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from getpass import getpass

os.environ['TELEGRAM_BOT_TOKEN'] = getpass('Enter your bot token: ')
TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')

# create the chatbot using pyTelegramBotAPI passing the token
bot = telebot.TeleBot(TOKEN)

Enter your bot token: ··········


In [4]:
# A Named Entity Recognition model for medication entities: medication name, dosage, duration, frequency, reason

tokenizer = AutoTokenizer.from_pretrained("samrawal/bert-large-uncased_med-ner")
model = AutoModelForTokenClassification.from_pretrained("samrawal/bert-large-uncased_med-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="SIMPLE")

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [24]:
# convert text into lowercase, break into words and check is a string of words containe "yes or "no"
def lowercase(text):
    return text.lower()

def is_yes(text):
    words = lowercase(text).split()
    return any(word in words for word in ["yes", "please", "yep", "yeah", "y", "ya", "sure", "ok", "yup", "yep", "maybe", "right"])

def is_no(text):
    words = lowercase(text).split()
    return any(word in words for word in ["no", "nope", "n", "nah", "not"])


# initalize bot state
state = 0

@bot.message_handler(func=lambda message: True)
def message(message):

    global state
    chat_id = message.chat.id

    if state == 0:
      bot.send_message(chat_id, "Hi there! Do you have any medication related entities to be recognised and extracted?")
      state = 1

    elif state == 1:
        if is_yes(message.text):
            bot.send_message(chat_id, "Please send me the text you want me to analyse.")
            state = 2
        elif is_no(message.text):
            bot.send_message(chat_id, "Ok, bye!")
            state = 0
        else:
            bot.send_message(chat_id, "I didn't get that. Please answer with yes or no.")
            state = 1

    elif state == 2:
      entities = nlp(message.text)

      if not entities or not set([entity["entity_group"] for entity in entities]) & set(['m', 'do', 'f', 'du']):
        bot.send_message(chat_id, f"No medication information found!")

      else:
        for entity in entities:

          if entity["entity_group"] == "m":
            medication = entity["word"].capitalize()

          if entity["entity_group"] == "do":
            dosage = entity["word"].capitalize()

          if entity["entity_group"] == "f":
            frequency = entity["word"]

          if entity["entity_group"] == "du":
            duration = entity["word"]

        bot.send_message(chat_id, f"Medication: {medication}\n Dosage: {dosage}\n Frequency: {frequency}\n Duration: {duration}")

      state = 3

    elif state == 3:
      bot.send_message(chat_id, "Do you have more medication related messages?")
      state = 1

bot.polling()